In [26]:
import pandas as pd
import numpy as np

# Corregir la ruta del archivo
df = pd.read_parquet("/home/jpcano/Donsson-Proyectos/API/ventashistoricas208semanas.parquet") #movimiento  facturas
df_p = pd.read_parquet("/home/jpcano/Donsson-Proyectos/API/costo_productos.parquet") #Costos unitarios
df_vp = pd.read_parquet("/home/jpcano/Donsson-Proyectos/API/ventas_perdidas_2025.parquet") #ventas perdidas
vp_reales = pd.read_excel("/home/jpcano/Donsson-Proyectos/INDICADOR NS/vp_agosto.xlsx") #vp reales


# EDA

## Facturas

In [27]:
import re
import unicodedata

# Diccionario de códigos a sucursales
mapa_codigos = {
    "FCAL": "SUCURSAL CALI",
    "FMED": "SUCURSAL MEDELLIN",
    "FMDE":"SUCURSAL MEDELLIN",
    "FCTG": "SUCURSAL CARTAGENA",
    "FBAQ": "SUCURSAL BARRANQUILLA",
    "FVAL": "SUCURSAL VALLADOLID",
    "FCOT":"PRINCIPAL COTA",
    "FBUC":"SUCURSAL BUCARAMANGA",
    "FNOR":"SUCURSAL NORTE",
    "FCL6":"SUCURSAL CALLE 6",
    "PV2E":"SUCURSAL CALLE 6",
    "PV3E":"SUCURSAL VALLADOLIR",
    "PV1E":"SUCURSAL COTA" ,#Las que comienzan por p son los mostradores
    "PV4E":"SUCURSAL NORTE",
    "PV9E":"SUCURSAL CALI"

}


# Equivalencias para normalizar nombres truncados o mal escritos
mapa_equivalencias = {
    "MEDELLIN": "SUCURSAL MEDELLIN",
    "MEDELLI": "SUCURSAL MEDELLIN",
    "MEDELL": "SUCURSAL MEDELLIN",
    "MEDELI": "SUCURSAL MEDELLIN",
    "CALI": "SUCURSAL CALI",
    "BUCARAMANGA":"SUCURSAL BUCARAMANGA",
    "BARRANQUILLA": "SUCURSAL BARRANQUILLA",
    "VALLADOLID": "SUCURSAL VALLADOLID",
    "CALLE 6":"SUCURSAL CALLE 6",
    "COTA":"PRINCIPAL COTA",
    "NORTE":"SUCURSAL NORTE"
}

def normalizar(texto):
    """Quita tildes y pasa a mayúsculas"""
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join([c for c in texto if not unicodedata.combining(c)])
    return texto.upper()

def extraer_sucursal(nombre):
    if not isinstance(nombre, str):
        return "VENDEDOR EXTERNO"
    
    sucursal = None
    
    # 1) Buscar "Mostrador ..."
    match = re.search(r"Mostrador\s+([A-Za-z0-9\s]+)", nombre, re.IGNORECASE)
    if match:
        sucursal = match.group(1).strip()
    else:
        # 2) Buscar "Calle" o "Cota"
        match2 = re.search(r"(Calle\s+\d+|Cota)", nombre, re.IGNORECASE)
        if match2:
            sucursal = match2.group(1).strip()
        else:
            # 3) Buscar prefijo de código
            for prefijo, ciudad in mapa_codigos.items():
                if nombre.upper().startswith(prefijo):
                    return ciudad
            return "VENDEDOR EXTERNO"
    
    # Normalizar texto
    sucursal = normalizar(sucursal)
    
    # Limpiar T1, T2, T3 al final
    sucursal = re.sub(r"\s*T\d+$", "", sucursal).strip()
    
    # Aplicar equivalencias
    sucursal = mapa_equivalencias.get(sucursal, sucursal)
    
    return sucursal

# Aplicar al dataframe
df["Sucursal"] = df["invoice_name"].apply(extraer_sucursal)

## Ventas perdidas

In [28]:
import pandas as pd
import numpy as np

# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("W").dt.start_time
df_vp["ano"]   = df_vp["fecha"].dt.year
df_vp["mes"]   = df_vp["fecha"].dt.month
df_vp["dia"]   = df_vp["fecha"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"], as_index=False)["ventas_perdidas"]
    .sum()
    .rename(columns={"ventas_perdidas": "lost_sales"})
)

# Mostrar resultado agrupado
vp_week = lost_by_week


In [ ]:
vp_reales["product_ref"] = vp_reales["Descripcion"].str.extract(r"\[([A-Z0-9]+)\]")
vp_reales.head()

# Asegurar que ambos son strings para evitar problemas
vp_week["product_ref"] = vp_week["product_ref"].astype(str)
vp_reales["product_ref"] = vp_reales["product_ref"].astype(str)

# 1. Obtener listas únicas
refs_week = set(vp_week["product_ref"].unique())
refs_real = set(vp_reales["product_ref"].unique())

# 2. Diferencia: los que están en vp_week pero no en vp_real
refs_extra = refs_week - refs_real

# 3. Filtrar el dataframe para verlos completos
df_discrepantes = vp_week[vp_week["product_ref"].isin(refs_extra)]


df_discrepantes = df_discrepantes[(df_discrepantes["mes"]==8) & (df_discrepantes["lost_sales"]>0) ]
df_discrepantes = df_discrepantes.groupby("product_ref").agg({"lost_sales":"sum"})
print("Cantidad de vp que no deberia tomar:", df_discrepantes["lost_sales"].sum())
df_discrepantes #Los productos que no se movieron hace mucho tiempo no salen en el analisis de ns

Cantidad de vp que no deberia tomar: 48.0


,lost_sales
product_ref,
DAB28118025,27.0
DAR12123UHE,2.0
DCS00342118,3.0
DCS00342186,1.0
DLS00105011,1.0
DLX00393020,14.0


In [30]:
vp_agosot_2025 = vp_week[(vp_week["ano"]==2025) & (vp_week["mes"]==8) ]
vp_agosot_2025.to_excel("vp_revisar.xlsx")

vp_agosot_2025.query("product_ref == 'DAB28118025' and store_name == 'PRINCIPAL COTA'")
#vp_agosot_2025.groupby("store_name")["lost_sales"].sum()

,store_name,product_ref,Semana,ano,mes,dia,lost_sales
6014,PRINCIPAL COTA,DAB28118025,2025-08-25,2025,8,26,2.0
6015,PRINCIPAL COTA,DAB28118025,2025-08-25,2025,8,27,2.0
6016,PRINCIPAL COTA,DAB28118025,2025-08-25,2025,8,28,1.0


## EMA SEMANAL MAS FUNCIONAL

In [31]:
import pandas as pd
import re
# --- 1. Crear columna de Semana ---
df["Semana"] = df["date_invoice"].dt.to_period("W").dt.start_time
df["Año"] = df["date_invoice"].dt.isocalendar().year
df["NroSemana"] = df["date_invoice"].dt.isocalendar().week

semana_ref = 35
anio_ref = 2025
num_semanas = 12

# lunes de la semana de referencia
fecha_ref = pd.to_datetime(f"{anio_ref}-W{semana_ref}-1", format="%G-W%V-%u")
fecha_inicio = fecha_ref - pd.Timedelta(weeks=num_semanas-1)

df["product_ref"] = df["product_name"].str.extract(r"\[(.*?)\]")



# filtrar
df_filtrado = df[(df["Semana"] >= fecha_inicio) & (df["Semana"] <= fecha_ref)]


ema_semanal = (
    df_filtrado.groupby(["Sucursal", "product_ref", "Semana"], as_index=False)
               .agg(quantity=("quantity", "sum"))
)

ema_semanal.columns=["store_name","product_ref","Semana","quantity"]
# HAY 8 DE LOS 10 EN MEDELLIN DAB28118025

# UNION

## EMA SEMANAL CON VP SEMANALES

In [32]:
df_merged = pd.merge(
    ema_semanal[["store_name", "product_ref", "Semana", "quantity"]],
    vp_week[["store_name", "product_ref", "Semana", "lost_sales","ano","mes"]],
    on=["store_name", "product_ref", "Semana"],
    how="outer"
).fillna(0)

df_merged["ventas_totales"] = df_merged["quantity"] + df_merged["lost_sales"]


In [33]:
df_merged.sample(10)

,store_name,product_ref,Semana,quantity,lost_sales,ano,mes,ventas_totales
18043,SUCURSAL BARRANQUILLA,BCS00231125,2025-02-24,0.0,2.0,2025.0,2.0,2.0
68429,SUCURSAL NORTE,DAE06544136,2025-01-06,0.0,1.0,2025.0,1.0,1.0
39508,SUCURSAL CALI,BCS00272125,2025-07-07,15.0,0.0,0.0,0.0,15.0
6408,PRINCIPAL COTA,BLS10047125,2025-08-11,4.0,0.0,0.0,0.0,4.0
29897,SUCURSAL BUCARAMANGA,BAB02034125,2024-12-02,0.0,1.0,2024.0,12.0,1.0
55287,SUCURSAL MEDELLIN,BAC00169125,2025-06-23,0.0,1.0,2025.0,6.0,1.0
51116,SUCURSAL CALLE 6,DAB18256025,2025-01-20,0.0,1.0,2025.0,1.0,1.0
36975,SUCURSAL BUCARAMANGA,DAR08147025,2025-06-16,0.0,1.0,2025.0,6.0,1.0
32709,SUCURSAL BUCARAMANGA,BHS00530125,2025-07-07,0.0,0.0,2025.0,7.0,0.0
15463,PRINCIPAL COTA,DLS00264189,2025-07-28,0.0,1.0,2025.0,7.0,1.0


In [34]:
# Convierte la columna 'Semana' a formato de fecha

df_vp_agosot_2025 = df_merged[(df_merged["ano"]==2025) & (df_merged["mes"]==8) ]

df_vp_agosot_2025.groupby("store_name")["lost_sales"].sum()

store_name
PRINCIPAL COTA           5902.0
SUCURSAL BARRANQUILLA    1635.0
SUCURSAL BUCARAMANGA      780.0
SUCURSAL CALI            1551.0
SUCURSAL CALLE 6         1354.0
SUCURSAL MEDELLIN        1010.0
SUCURSAL NORTE            517.0
SUCURSAL VALLADOLID       782.0
Name: lost_sales, dtype: float64

In [35]:
# --- 6. Función para calcular EMA con EMA0 = promedio primeras 4 semanas ---
def calcular_ema(grupo, alpha=0.2):
    grupo = grupo.sort_values("Semana").reset_index(drop=True)
    # EMA inicial: promedio primeras 4 semanas (si hay menos, promedio todas)
    ema0 = grupo.loc[:3, "quantity"].mean() if len(grupo) >= 4 else grupo["quantity"].mean() #ema [0] parte de las fechas como se tomaron luego organiza el numero de las semanas la consulta es , para incluir las ventas perdidas hay que hacerlo a nivel de seman con los condicionales del docuemnto 
    ema_vals = [ema0]
    for t in range(1, len(grupo)):
        xt = grupo.loc[t, "quantity"]
        ema_vals.append(alpha * xt + (1 - alpha) * ema_vals[-1]) #
    grupo["EMA"] = ema_vals
    return grupo

# --- 7. Aplicar EMA ---
df_semanal = (
    df_semanal.groupby(["Sucursal", "product_name"], group_keys=False)
              .apply(calcular_ema)
)

# --- 8. Añadir año y número de semana ISO ---
df_semanal["Año"] = df_semanal["Semana"].dt.isocalendar().year
df_semanal["NroSemana"] = df_semanal["Semana"].dt.isocalendar().week

# --- 9. Filtrar para ver el resultado de la semana de referencia ---
#df_final = df_semanal[df_semanal["NroSemana"] == 35]

NameError: name 'df_semanal' is not defined